In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
lr=0.001
training_iters=100000
batch_size=128
#display_step=10

n_inputs=28
n_steps=28
n_hidden_units=128
n_classes=10

In [4]:
x=tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y=tf.placeholder(tf.float32,[None,n_classes])

In [9]:
w={
    'in':tf.Variable(tf.random_normal([n_inputs,n_hidden_units])),
    'out':tf.Variable(tf.random_normal([n_hidden_units,n_classes]))
}
b={
    'in':tf.Variable(tf.constant(0.1,shape=[n_hidden_units,])),
    'out':tf.Variable(tf.constant(0.1,shape=[n_classes,]))
}

In [16]:
def RNN(X,w,b):
    X=tf.reshape(X,[-1,n_inputs])
    X_in=tf.matmul(X,w['in'])+b['in']
    X_in=tf.reshape(X_in,[-1,n_steps,n_hidden_units])
    
    lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(n_hidden_unis,forget_bias=1.0,state_is_tuple=True)
    _init_state=lstm_cell.zero_state(batch_size,dtype=tf.float32)
    outputs,states=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=_init_state,time_major=False)
    
    results=tf.matmul(states[1],w['out'])+b['out']
    ##or
    ##unpack to list[(batch,outputs)...]*steps
    #outputs=tf.unpack(tf.transpose(outputs,[1,0,2]))
    #results=tf.matmul(outputs[-1],w['out'])+b['out']
    return results

In [18]:
pred=RNN(x,w,b)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=pred))
train_op=tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-10-890cdfbd585d>", line 8, in RNN
    outputs,states=tf.nn.dynamic_rnn(lstm_cell,X_in,initial_state=_init_state,time_major=False)
  File "<ipython-input-11-45cf580a3dae>", line 1, in <module>
    pred=RNN(x,w,b)
  File "/home/fzyt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
init=tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    step=0
    while step*batch_size<training_iters:
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        batch_xs=batch_xs.reshape([batch_size,n_steps,n_inputs])
        sess.run([train_op],feed_dict={
            x:batch_xs,
            y:batch_ys,
        })
        if step %20 ==0:
            print(sess.run(accuracy,feed_dict={
                x:batch_xs,
                y:batch_ys,   
            }))
        step+=1

Instructions for updating:
Use `tf.global_variables_initializer` instead.


NameError: name 'train_op' is not defined